In [1]:
import re
from bs4 import BeautifulSoup
import datetime as dt
from urllib.request import urlopen
from db_utils import retrieve_data, generate_unique_game_id, get_rank_from_row, load_config
from scrape_all_games_data import get_daily_games_at_date
from io import StringIO
import pandas as pd
import sys
from scrape_quarters_data import get_quarters_data_at_date_home, insert_quarters_data_at_date_home
import os
os.chdir("../../../..")
os.getcwd()

'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting'

In [3]:
# date = dt.date(2024, 9, 20)
# home = "Washington State"
# insert_quarters_data_at_date_home(date, home)

In [ ]:
query = """
    INSERT INTO cfb.all_games_log(unique_id, date)
    FROM cfb.all_games
"""

In [2]:
# TODO: redo all the unique ID'
# sys.path.insert(0, "../../..")
query = """
    SELECT *
    FROM cfb.all_games
"""
game_df = retrieve_data(query)
game_df2 = game_df

In [3]:
game_df.rename(columns={'home': 'Home', 'visitor': 'Visitor', 'date': 'Date'}, inplace=True)

In [4]:
game_df2["unique_id"] = game_df.apply(generate_unique_game_id, axis=1)

In [5]:
import psycopg2

In [8]:
game_df2.rename(columns={'Home': 'home', 'Visitor': 'visitor', 'Date': 'date'}, inplace=True)

In [10]:
config = load_config()
db_config = config["database"]
conn = psycopg2.connect(**db_config)
cur = conn.cursor()

# SQL Insert Query
insert_query = """
    INSERT INTO cfb.all_games_log (unique_id, date)
    VALUES (%s, %s);
"""

# Loop through DataFrame and insert each row
for _, row in game_df2.iterrows():
    # Extract column values from each row
    data = (row['unique_id'], row['date'])
    
    # Execute the insert query
    cur.execute(insert_query, data)

# Commit all changes
conn.commit()
print("Data inserted successfully!")

# Close connection
cur.close()
conn.close()

Data inserted successfully!


In [9]:
config = load_config()
db_config = config["database"]
conn = psycopg2.connect(**db_config)
cur = conn.cursor()

update_query = """
    UPDATE cfb.all_games
    SET unique_id = %s
    WHERE home = %s AND visitor = %s AND date = %s;
"""

# Loop through DataFrame and update each row
for _, row in game_df2.iterrows():
    cur.execute(update_query, (row["unique_id"], row["home"], row["visitor"], row["date"]))
    print(f"updated {row['home']} {row['date']}")

# Commit all updates
conn.commit()
print("Updates successful!")

# Close connection
cur.close()
conn.close()

updated Colorado State 2015-10-10
updated Eastern Michigan 2015-10-10
updated Florida Atlantic 2015-10-10
updated Florida International 2015-10-10
updated Florida State 2015-10-10
updated Fresno State 2015-10-10
updated Georgia State 2015-10-10
updated Hawaii 2015-10-10
updated Iowa 2015-10-10
updated Kansas 2015-10-10
updated Kansas State 2015-10-10
updated Louisiana 2015-10-10
updated Michigan 2015-10-10
updated Ole Miss 2015-10-10
updated Mississippi State 2015-10-10
updated Missouri 2015-10-10
updated Nebraska 2015-10-10
updated Nevada 2015-10-10
updated UNLV 2015-10-10
updated North Texas 2015-10-10
updated Northern Illinois 2015-10-10
updated Notre Dame 2015-10-10
updated Ohio 2015-10-10
updated Ohio State 2015-10-10
updated Oregon 2015-10-10
updated Penn State 2015-10-10
updated Pitt 2015-10-10
updated Purdue 2015-10-10
updated Rutgers 2015-10-10
updated South Carolina 2015-10-10
updated South Florida 2015-10-10
updated Temple 2015-10-10
updated Air Force 2024-11-16
updated Alab

In [3]:
get_quarters_data_at_date_home(dt.date(2025, 1, 1), "Arizona State").columns

Index(['visitor_first_quarter', 'visitor_second_quarter',
       'visitor_third_quarter', 'visitor_fourth_quarter', 'visitor_ot_total',
       'OT', 'home_first_quarter', 'home_second_quarter', 'home_third_quarter',
       'home_fourth_quarter', 'home_ot_total', 'date', 'home', 'visitor',
       'unique_id'],
      dtype='object')

In [3]:
url = "https://www.sports-reference.com/cfb/boxscores/2025-01-20-notre-dame.html"
html = urlopen(url)
soup = BeautifulSoup(html, features="html.parser")
cand_divs = soup.find_all(attrs={"class": re.compile("linescore nohover")})

In [4]:
html_io = StringIO(str(cand_divs[0]))
df = pd.read_html(html_io)[0]

In [5]:
# double check OT formatting.
visitor_row = df.iloc[1]
visitor_row["Unnamed: 1"].replace(u'\xa0', ' ')
visitor = get_rank_from_row(visitor_row, "Unnamed: 1")[0]

In [6]:
df

,Unnamed: 0,Unnamed: 1,1,2,3,4,Final
0,via Sports Logos.net About logos,Ohio State (6),0,21,10,3,34
1,via Sports Logos.net About logos,Notre Dame (3),7,0,8,8,23


In [7]:
df.columns[-2]

'4'

In [8]:
team = ["visitor", "home"]
quarter = ["first_quarter", "second_quarter", "third_quarter", "fourth_quarter"]
col_dict = {}
for i in range(2):
    for j in range(4):
        col = f"{team[i]}_{quarter[j]}"
        # Map the column to the data (indexed from 1)
        col_dict[col] = int(df[str(j+1)][i])
    # If we go into OT
    if df.columns[-2] != "4":
        assert df.columns[-2].isnumeric(), "DataFrame for quarters in wrong format"
        ot_total = 0
        # If there are 3 OT's, then columns up to 7, want to sum 5 through 7
        for k in range(5, int(df.columns[-2])+1):
            ot_total += df[str(k)][i]
        ot_col = f"{team[i]}_ot_total"
        col_dict[ot_col] = ot_total
col_df = pd.DataFrame([col_dict])

In [12]:
df.iloc[0,-1:]

Final    34
Name: 0, dtype: object

In [40]:
col_df

,visitor_first_quarter,visitor_second_quarter,visitor_third_quarter,visitor_fourth_quarter,home_first_quarter,home_second_quarter,home_third_quarter,home_fourth_quarter
0,0,21,10,3,7,0,8,8


In [20]:
df

,Unnamed: 0,Unnamed: 1,1,2,3,4,Final
0,via Sports Logos.net About logos,Ohio State (6),0,21,10,3,34
1,via Sports Logos.net About logos,Notre Dame (3),7,0,8,8,23


In [ ]:
import datetime as dt
# import pandas as pd
# import psycopg2
# import pytz
# import re
# import sys
# import time
# import warnings
# from bs4 import BeautifulSoup
# from io import StringIO
# from psycopg2.extras import execute_values
# from psycopg2.extensions import AsIs
# from typing import List
# from urllib.request import urlopen
# import sys
# # from src.db_utils import generate_unique_game_id, execute_sql_script
# from sports_betting.src.cfb.data.scrape_all_games_data import get_daily_games_at_date, clean_df, insert_daily_games_at_date, backfill_games
# from typing import Callable
# from db_utils import load_config, retrieve_data

In [1]:
import sys
print(sys.path)

['C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv', '', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages', 'C:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\src', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\win32', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\Pythonwin']


In [3]:
import db_utils

ModuleNotFoundError: No module named 'pandas'

In [2]:
from pip import _internal
_internal.main(['list'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Package           Version

----------------- -----------

asttokens         3.0.0

colorama          0.4.6

...

stack-data        0.6.3

tornado           6.4.2

traitlets         5.14.3

wcwidth           0.2.13

0

In [3]:
import sports_betting

ModuleNotFoundError: No module named 'sports_betting'

In [3]:

import sys
print(sys.path)

['C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\python312.zip', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\DLLs', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312\\Lib', 'C:\\Users\\brtnl\\AppData\\Local\\Programs\\Python\\Python312', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv', '', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\win32', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\brtnl\\OneDrive\\Desktop\\code\\sports_betting\\venv\\Lib\\site-packages\\Pythonwin']


In [6]:
pip -V

pip 24.3.1 from c:\Users\brtnl\OneDrive\Desktop\code\sports_betting\venv\Lib\site-packages\pip (python 3.12)

Note: you may need to restart the kernel to use updated packages.


In [6]:
import sports_betting

ModuleNotFoundError: No module named 'sports_betting'

In [ ]:
def get_rank_from_row(row):
    pattern2 = r"^(.*?)\s\((\d{1,2})\)$"
    university = row[0]
    match = re.fullmatch(pattern2, university)
    if match:
         return match.group(1), int(match.group(2))
    else:
         return university, None

In [ ]:
get_rank_from_row(all_games_table[5].iloc[2])

('Arizona', '21')

In [ ]:
all_games_table[5]

,0,1,2
0,Sat 8/31,Sat 8/31,Sat 8/31
1,New Mexico,39,Final
2,Arizona (21),61,NaN


In [ ]:
temp.iloc[0][0]

'Western Kentucky'

In [ ]:
regex2 = r".*\(\b([1-9]|1[0-9]|2[0-5])\b\)"
re.fullmatch(regex2, "Western Kentucky (5)")

<re.Match object; span=(0, 20), match='Western Kentucky (5)'>

In [ ]:
temp = all_games_table[3][1:]
temp.loc[:, "rank"] = None
# temp

/var/folders/z_/5kxjclhd5090czxnlnxslydr0000gn/T/ipykernel_88211/1111760236.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:, "rank"] = None


regex = "^[A-Za-z]{3} \d{1,2}/\d{2}$"
string = all_games_table[3].iloc[0][0]

s = re.fullmatch(regex, string)

In [ ]:
s

<re.Match object; span=(0, 8), match='Sat 8/31'>

In [ ]:
month = month.lower()

    # URL to scrape, notice f string:
    url = (
        f"https://www.basketball-reference.com/leagues/NBA_%(year)s_games-%(month)s.html"
        % {"year": year, "month": month}
    )
    try:
        html = urlopen(url)
    except Exception as e:
        print(e)
        print(
            "Failure: %(month)s, %(year)s all_games URL not pulled"
            % {"month": month.capitalize(), "year": year}
        )
        return None

    # Kicks you out if you request over 20 times over a minute
    time.sleep(3.5)

    soup = BeautifulSoup(html, features="html.parser")

    # Convert the tables into a string and wrap it with StringIO
    tables = soup.find_all("table", {"id": re.compile("schedule")})
    html_string = "\n".join(str(table) for table in tables)
    html_io = StringIO(html_string)

    all_games_table = pd.read_html(html_io, header=0)[0]
    all_games_table = clean_all_games_table(all_games_table)
    if all_games_table is not None:
        all_games_table["Unique_ID"] = all_games_table.apply(
            generate_unique_game_id, axis=1
        )
    return all_games_table